# Groupby_Apply

In [18]:
import subprocess
import os

import pandas as pd
import numpy as np
import requests

from datetime import datetime

from bs4 import BeautifulSoup


%matplotlib inline


import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

import plotly.graph_objects as go

In [19]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,"Korea, South",1.0
2,2020-01-22,no,Kosovo,0.0
3,2020-01-22,no,Kuwait,0.0
4,2020-01-22,no,Kyrgyzstan,0.0


# test_data

In [20]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Germany'))&
                      (pd_JH_data['date']>'2020-03-20')]
test_data.head()

,date,state,country,confirmed
15736,2020-03-21,no,Germany,22213.0
15791,2020-03-21,no,US,25825.0
16002,2020-03-22,no,Germany,24873.0
16043,2020-03-22,no,US,33761.0
16268,2020-03-23,no,Germany,29056.0


In [21]:
test_data

,date,state,country,confirmed
15736,2020-03-21,no,Germany,22213.0
15791,2020-03-21,no,US,25825.0
16002,2020-03-22,no,Germany,24873.0
16043,2020-03-22,no,US,33761.0
16268,2020-03-23,no,Germany,29056.0
...,...,...,...,...
51435,2020-08-02,no,US,4668172.0
51647,2020-08-03,no,Germany,212111.0
51688,2020-08-03,no,US,4713540.0
51912,2020-08-04,no,Germany,212828.0


In [22]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-08-04,no,212828.0
US,2020-08-04,no,4771080.0


In [26]:
# %load ../src/features/build_features.py

import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function (data structure kept)

        parameters:
        ----------
        df_input : pandas.series
        column : str
        window : int
            used data points to calculate the filter result

        Returns:
        ----------
        df_result: pd.DataFrame
            the index of the df_input has to be preserved in result
    '''

    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0) # attention with the neutral element here

    result=signal.savgol_filter(np.array(filter_in),
                           window, # window size used for filtering
                           1)
    df_result[str(column+'_filtered')]=result
    return df_result

def rolling_reg(df_input,col='confirmed'):
    ''' Rolling Regression to approximate the doubling time'

        Parameters:
        ----------
        df_input: pd.DataFrame
        col: str
            defines the used column
        Returns:
        ----------
        result: pd.DataFrame
    '''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)



    return result




def calc_filtered_data(df_input,filter_on='confirmed'):
    '''  Calculate savgol filter and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['state','country',filter_on]].groupby(['state','country']).apply(savgol_filter)#.reset_index()

    #print('--+++ after group by apply')
    #print(pd_filtered_result[pd_filtered_result['country']=='Germany'].tail())

    #df_output=pd.merge(df_output,pd_filtered_result[['index',str(filter_on+'_filtered')]],on=['index'],how='left')
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    #print(df_output[df_output['country']=='Germany'].tail())
    return df_output.copy()





def calc_doubling_rate(df_input,filter_on='confirmed'):
    ''' Calculate approximated doubling rate and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'


    pd_DR_result= df_input.groupby(['state','country']).apply(rolling_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output


if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

    pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
    pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()

    #test_structure=pd_JH_data[((pd_JH_data['country']=='US')|
    #                  (pd_JH_data['country']=='Germany'))]

    pd_result_larg=calc_filtered_data(pd_JH_data)
    pd_result_larg=calc_doubling_rate(pd_result_larg)
    pd_result_larg=calc_doubling_rate(pd_result_larg,'confirmed_filtered')


    mask=pd_result_larg['confirmed']>100
    pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)
    pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)
    print(pd_result_larg[pd_result_larg['country']=='Germany'].tail())


the test slope is: [2.]
            date state  country  confirmed  confirmed_filtered  confirmed_DR  \
28415 2020-07-31    no  Germany   210399.0            210141.0    226.112610   
28416 2020-08-01    no  Germany   211005.0            210854.0    286.140136   
28417 2020-08-02    no  Germany   211220.0            211512.6    513.701989   
28418 2020-08-03    no  Germany   212111.0            212109.0    382.360458   
28419 2020-08-04    no  Germany   212828.0            212705.4    263.747512   

       confirmed_filtered_DR  
28415             282.761365  
28416             296.898088  
28417             307.430543  
28418             337.038831  
28419             355.648893  


In [27]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state country                      
no    Germany 2020-08-04   212828.0
      US      2020-08-04  4771080.0

In [28]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [29]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country       
no     Germany  15736           NaN
                16002           NaN
                16268      7.417994
                16535      7.142035
                16800      8.012983
                            ...    
       US       50889     66.526569
                51155     72.612628
                51435     87.059011
                51688    100.431094
                51954     91.685726
Name: confirmed, Length: 274, dtype: float64

In [30]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [31]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR',
                             'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,266,NaN
2,Alberta,Canada,532,NaN
3,Alberta,Canada,798,NaN
4,Alberta,Canada,1064,NaN


In [36]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,level_0,index,date,state,country,confirmed
0,0,0,2020-01-22,Alberta,Canada,0.0
1,1,169,2020-01-22,no,"Korea, South",1.0
2,2,170,2020-01-22,no,Kosovo,0.0
3,3,171,2020-01-22,no,Kuwait,0.0
4,4,172,2020-01-22,no,Kyrgyzstan,0.0


In [35]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()


,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,169,2020-01-22,no,"Korea, South",1.0,NaN
2,170,2020-01-22,no,Kosovo,0.0,NaN
3,171,2020-01-22,no,Kuwait,0.0,NaN
4,172,2020-01-22,no,Kyrgyzstan,0.0,NaN


# Filtering with data by groupbyapply

In [38]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result

In [39]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [40]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0
1,169,2020-01-22,no,"Korea, South",1.0,NaN,-4.8
2,170,2020-01-22,no,Kosovo,0.0,NaN,0.0
3,171,2020-01-22,no,Kuwait,0.0,NaN,0.0
4,172,2020-01-22,no,Kyrgyzstan,0.0,NaN,10.8


# Filtering doubling rate

In [41]:
pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
52131,no,Zimbabwe,50938,163.645589
52132,no,Zimbabwe,51136,181.446357
52133,no,Zimbabwe,51433,4.397768
52134,no,Zimbabwe,51668,5.012365
52135,no,Zimbabwe,52135,-1.646440


In [42]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
52131,51964,2020-08-04,no,Barbados,132.0,inf,10.0,-2.381491
52132,51965,2020-08-04,no,Belarus,68250.0,744.928962,4821.0,0.999371
52133,51966,2020-08-04,no,Belgium,70648.0,175.895703,191.8,1.021802
52134,51952,2020-08-04,no,Albania,5750.0,48.741703,1581.6,0.724016
52135,52135,2020-08-04,no,Zimbabwe,4221.0,27.148889,4313.0,-1.646440


In [43]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)

In [44]:
pd_result_larg[pd_result_larg['country']=='Germany'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
50849,50950,2020-07-31,no,Germany,210399.0,226.112610,269.0,204.447655
51115,51216,2020-08-01,no,Germany,211005.0,286.140136,2971.2,1.597543
51380,51482,2020-08-02,no,Germany,211220.0,513.701989,2983.4,-1.642711
51647,51748,2020-08-03,no,Germany,212111.0,382.360458,269.0,244.352510
51912,52014,2020-08-04,no,Germany,212828.0,263.747512,269.0,176.663100


In [45]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)